In [1]:
import json
import os
import time
import pickle

import seaborn as sns
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

import importlib

In [2]:
df = pd.read_json("~/Downloads/paper_data.json")

In [4]:
df['decision'] = ['Accept', 'Accept', 'Reject', 'Accept']

In [6]:
def expand_reviews(df):
    entries = []
    for i, row in df.iterrows():
        reviews = row['reviews']
        n = len(reviews)
        for review in row['reviews']:
            entries.append({'title': row['title'], 'decision': row['decision'], 
                            'conference': row['conference'], 'review': review, 'text': row['text']})
    return pd.DataFrame(entries)

In [8]:
df = expand_reviews(df)

In [11]:
df = pd.concat([df for i in range(10)])

In [15]:
df = df.reset_index(drop=True)

In [18]:
df.to_json('/Users/erichansen/expanded_papers.json', orient='records')

In [17]:
df

,title,decision,conference,review,text
0,"Wider and Deeper, Cheaper and Faster: Tensoriz...",Accept,NIPS,The paper explores a new way to share paramete...,We consider the time-series prediction task of...
1,"Wider and Deeper, Cheaper and Faster: Tensoriz...",Accept,NIPS,The paper introduces a tensorized version of L...,We consider the time-series prediction task of...
2,"Wider and Deeper, Cheaper and Faster: Tensoriz...",Accept,NIPS,\nThis paper proposes Tensorized LSTMs for eff...,We consider the time-series prediction task of...
3,Concentration of Multilinear Functions of the ...,Accept,NIPS,This paper discusses concentration of bilinear...,"Statistical Physics, the Ising model has found..."
4,Concentration of Multilinear Functions of the ...,Accept,NIPS,Summary:\n \n The paper considers concentrati...,"Statistical Physics, the Ising model has found..."
...,...,...,...,...,...
1095,Deep Subspace Clustering Networks,Reject,NIPS,The paper addresses the problem of subspace cl...,Subspace clustering has become an important pr...
1096,Deep Subspace Clustering Networks,Reject,NIPS,This paper proposes a new subspace clustering ...,Subspace clustering has become an important pr...
1097,Attentional Pooling for Action Recognition,Accept,NIPS,This paper presents an attention-based model f...,"Interestingly, even video based action recogni..."
1098,Attentional Pooling for Action Recognition,Accept,NIPS,The paper introduces a simple yet efficient wa...,"Interestingly, even video based action recogni..."


In [6]:
! pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0
    Uninstalling pip-23.0:
      Successfully uninstalled pip-23.0


In [2]:
! pip install datasets | grep -v 'already satisfied'
! pip install sentencepiece | grep -v 'already satisfied'
! pip install rouge_score | grep -v 'already satisfied'
! pip install wandb | grep -v 'already satisfied'
! pip install bert-score | grep -v 'already satisfied'
! pip install evaluate | grep -v 'already satisfied'
! pip install transformers -U  | grep -v 'already satisfied'

In [3]:
sublibraries = [f'ICLR{year}' for year in range(2017, 2021)] + [f'NIPS{year}' for year in range(2017, 2021)] 

# Data Extraction

# Dataset Pre-Processing

## Load dataset

In [5]:
workdir = './'

In [7]:
from datasets import concatenate_datasets, DatasetDict, load_dataset
from sklearn.model_selection import train_test_split

df = pd.read_json(workdir + "paper_data.json")

# train_df, test_df = train_test_split(df, test_size=0.25, random_state=100)

# paper_reviews_dataset = DatasetDict({
#     "train": train_df,
#     "test": test_df
# })

# paper_reviews_dataset = load_dataset("json", data_files="/Users/erichansen/Downloads/paper_data.json")
paper_reviews_dataset = load_dataset("json", data_files='paper_data.json')
paper_reviews_dataset = paper_reviews_dataset['train'].train_test_split(test_size=0.25)

Using custom data configuration default-d17c992324baeca4


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /Users/cynthiachen/.cache/huggingface/datasets/json/default-d17c992324baeca4/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
df

,title,decision,conference,text,reviews
0,"Wider and Deeper, Cheaper and Faster: Tensoriz...",Accept,NIPS,We consider the time-series prediction task of...,[The paper explores a new way to share paramet...
1,Concentration of Multilinear Functions of the ...,Accept,NIPS,"Statistical Physics, the Ising model has found...",[This paper discusses concentration of bilinea...
2,Deep Subspace Clustering Networks,Accept,NIPS,Subspace clustering has become an important pr...,[\nThe paper propose to enforce self-expressiv...
3,Attentional Pooling for Action Recognition,Accept,NIPS,"Interestingly, even video based action recogni...",[This paper presents an attention-based model ...


## Tokenize dataset

In [131]:
max_paper_extract_length = 512
max_review_length = 512
pre_trained_model_checkpoint = "facebook/bart-large-cnn"


In [132]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(pre_trained_model_checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/erichansen/.cache/huggingface/hub/models--facebook--bart-large-cnn/snapshots/3d224934c6541b2b9147e023c2f6f6fe49bd27e1/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_g

In [133]:
def preprocess_function(paper_json):
    print('reached')
    model_inputs = tokenizer(
        paper_json["text"],
        max_length=max_paper_extract_length,
        truncation=True,
    )
    paper_json['review'] = paper_json['reviews'][0]
    labels = tokenizer(
        paper_json["review"], max_length=max_review_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    print(len(model_inputs['labels']))
    print(len(paper_json['text']))
    return model_inputs

# Model Training

## Load Pre-Trained Bart Model

In [135]:
from transformers import BartForConditionalGeneration, AutoModelForSeq2SeqLM
model = BartForConditionalGeneration.from_pretrained(model_checkpoint)

loading configuration file config.json from cache at /Users/erichansen/.cache/huggingface/hub/models--facebook--bart-large-cnn/snapshots/3d224934c6541b2b9147e023c2f6f6fe49bd27e1/config.json
Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL

## Fine-Tune Bart Model using Tokenized, Extracted Training Set

### Evaluation Metrics

In [158]:
import evaluate
from evaluate import load
import bert_score

# bertscore = load("bertscore")
# predictions = ["hello there", "general kenobi"]
# references = ["hello there", "general kenobi"]

predictions = ["hello there young padawan", "general obi wan "]
references = ["hello there youngster", "general kenobi"]

results = bert_score.score(predictions, references, lang='en', verbose=True)
print(results)

rouge_score = evaluate.load("rouge")
scores = rouge_score.compute(
    predictions=predictions, references=references
)
scores

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/erichansen/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /Users/erichanse

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.15 seconds, 12.92 sentences/sec
(tensor([0.9222, 0.8905]), tensor([0.9714, 0.8988]), tensor([0.9462, 0.8946]))


{'rouge1': 0.48571428571428577,
 'rouge2': 0.2,
 'rougeL': 0.48571428571428577,
 'rougeLsum': 0.48571428571428577}

In [167]:
import numpy as np
import nltk
from nltk import sent_tokenize
import bert_score


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    rouge_results = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Compute BERT Score, rescaling with baseline as specified in the Ti
    # https://github.com/Tiiiger/bert_score/blob/master/example/Demo.ipynb
    # https://arxiv.org/pdf/1904.09675.pdf
    bert_scores = bert_score.score(decoded_preds, decoded_labels, lang='en', 
                                   verbose=False, rescale_with_baseline=True)
    bs_prec, bs_recall, bs_f1 = bert_scores[0].mean(), bert_scores[1].mean(), bert_scores[2].mean()

#     bert_score_results = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang='en')
    # Extract the median scores
    results = {k: round(v, 4) for k, v in rouge_results.items()}
    
    # Only report F1 Score, similar to the Yuan et. al 
    results.update({'BertScore': bs_f1})
#     result.update({})
    return results

### Training Arguments

In [138]:
from transformers import Seq2SeqTrainingArguments

batch_size = 1
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_dataset["train"]) // batch_size 
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-review-advisor",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
#     push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


### Data Collator
Necessary for HuggingFace Batching of Feature Data

In [139]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

### Training Function

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: conference, decision, review, title, reviews, text. If conference, decision, review, title, reviews, text are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/Users/erichansen/.conda/envs/data-sci/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3
  Num Epochs = 8
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 24
  Number of trainable parameters = 406290432
Automatic Weights & Biases lo

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bertscore
1,0.367100,6.438890,0.324200,0.044200,0.153800,0.274700,0.010323
2,0.359800,6.768425,0.342400,0.027300,0.157600,0.315200,0.049732


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: conference, decision, review, title, reviews, text. If conference, decision, review, title, reviews, text are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1
  Batch size = 1
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_length": 142,
  "min_length": 56,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/erichansen/.cache/huggingface/hub/models--roberta-large/snapshots/5069

Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /Users/erichansen/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/pytorch_model.bin
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer

## Evaluate Model

In [ ]:
trainer.evaluate()

# Reproduce Results

In [ ]:
## Load Data 